<a href="https://colab.research.google.com/github/abequette/CS167Fall22Code/blob/main/Day05_Notes_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 5 Notes: k Nearest Neighbors
The following code includes the steps necessary to build a k nearest neighbors model. We'll go through each step, and then I'll ask you to pull these steps together into a function that you can pass a new observation of a flower in to and it will output the species the model predicts that flower is.

In [30]:
import pandas
from google.colab import drive
drive.mount('/content/drive')
#go through authentication step

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import the data and take a look at it
iris_data = pandas.read_csv('/content/drive/MyDrive/CS167/irisData.csv')
iris_data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
#compare a series of booleans vs. a dataframe with rows from the boolean values
iris_data['sepal length'] < 5.0

In [3]:
iris_data[iris_data['sepal length'] < 5.0]

,sepal length,sepal width,petal length,petal width,species
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa
11,4.8,3.4,1.6,0.2,Iris-setosa
12,4.8,3.0,1.4,0.1,Iris-setosa
13,4.3,3.0,1.1,0.1,Iris-setosa
22,4.6,3.6,1.0,0.2,Iris-setosa


In [31]:
# Calculate the distance
import numpy
#adding a new column
iris_data['distance_to_new'] = numpy.sqrt(
    (5.1-iris_data['petal length'])**2
    +(7.2-iris_data['sepal length'])**2
    +(1.5-iris_data['petal width'])**2
    +(2.5-iris_data['sepal width'])**2)
iris_data

,sepal length,sepal width,petal length,petal width,species,distance_to_new
0,5.1,3.5,1.4,0.2,Iris-setosa,4.559605
1,4.9,3.0,1.4,0.2,Iris-setosa,4.573839
2,4.7,3.2,1.3,0.2,Iris-setosa,4.782259
3,4.6,3.1,1.5,0.2,Iris-setosa,4.665833
4,5.0,3.6,1.4,0.2,Iris-setosa,4.629255
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica,1.072381
146,6.3,2.5,5.0,1.9,Iris-virginica,0.989949
147,6.5,3.0,5.2,2.0,Iris-virginica,1.000000
148,6.2,3.4,5.4,2.3,Iris-virginica,1.593738


In [5]:
# Sort the data and display it
sorted_data = iris_data.sort_values(['distance_to_new']) 
sorted_data #display it

,sepal length,sepal width,petal length,petal width,species,distance_to_new
76,6.8,2.8,4.8,1.4,Iris-versicolor,0.591608
52,6.9,3.1,4.9,1.5,Iris-versicolor,0.700000
77,6.7,3.0,5.0,1.7,Iris-versicolor,0.741620
50,7.0,3.2,4.7,1.4,Iris-versicolor,0.836660
129,7.2,3.0,5.8,1.6,Iris-virginica,0.866025
...,...,...,...,...,...,...
8,4.4,2.9,1.4,0.2,Iris-setosa,4.835287
38,4.4,3.0,1.3,0.2,Iris-setosa,4.921382
42,4.4,3.2,1.3,0.2,Iris-setosa,4.945705
22,4.6,3.6,1.0,0.2,Iris-setosa,5.144900


In [6]:
# give the top 5 closest neighbors
sorted_data.iloc[0:5]

,sepal length,sepal width,petal length,petal width,species,distance_to_new
76,6.8,2.8,4.8,1.4,Iris-versicolor,0.591608
52,6.9,3.1,4.9,1.5,Iris-versicolor,0.700000
77,6.7,3.0,5.0,1.7,Iris-versicolor,0.741620
50,7.0,3.2,4.7,1.4,Iris-versicolor,0.836660
129,7.2,3.0,5.8,1.6,Iris-virginica,0.866025


In [8]:
# give the most commonly occuring species of the 5 closest neighbors
#sorted_data.iloc[0:5]['species'].mode()
sorted_data.iloc[0:5]['species'].mode()[0] #return just the string

'Iris-versicolor'

## Exercise:
Now it's your turn. Use the code snippets from above to build out a kNN function that takes in `specimen`, `data`, and `k` and prints out the prediction. 

In [37]:
def knn(specimen, training_data, k):
  data_copy = training_data.copy()
  data_copy['distance_to_new'] = numpy.sqrt((specimen['petal length']-data_copy['petal length'])**2
  + (specimen['sepal length']-data_copy['sepal length'])**2
  + (specimen['petal width']-data_copy['petal width'])**2
  + (specimen['sepal width']-data_copy['sepal width'])**2)
  sorted_new_data = data_copy.sort_values(['distance_to_new'])
  top_k = sorted_new_data.iloc[0:k] 
  return top_k['species'].mode()[0]
  


  

In [38]:
# Use the code below to build a dictionary of values and test your new function:
unknown_iris = {}
unknown_iris['petal length'] = 5.1
unknown_iris['sepal length'] = 7.2
unknown_iris['petal width'] = 1.5
unknown_iris['sepal width'] = 2.5
# 5-NN 
for k in range(1,50):
  print(k, knn(unknown_iris, iris_data,k))

1 Iris-versicolor
2 Iris-versicolor
3 Iris-versicolor
4 Iris-versicolor
5 Iris-versicolor
6 Iris-versicolor
7 Iris-versicolor
8 Iris-versicolor
9 Iris-versicolor
10 Iris-versicolor
11 Iris-versicolor
12 Iris-versicolor
13 Iris-versicolor
14 Iris-versicolor
15 Iris-versicolor
16 Iris-versicolor
17 Iris-virginica
18 Iris-virginica
19 Iris-virginica
20 Iris-virginica
21 Iris-virginica
22 Iris-virginica
23 Iris-virginica
24 Iris-virginica
25 Iris-virginica
26 Iris-virginica
27 Iris-virginica
28 Iris-virginica
29 Iris-virginica
30 Iris-virginica
31 Iris-virginica
32 Iris-virginica
33 Iris-virginica
34 Iris-virginica
35 Iris-virginica
36 Iris-virginica
37 Iris-virginica
38 Iris-virginica
39 Iris-virginica
40 Iris-virginica
41 Iris-virginica
42 Iris-virginica
43 Iris-virginica
44 Iris-virginica
45 Iris-virginica
46 Iris-virginica
47 Iris-virginica
48 Iris-virginica
49 Iris-virginica


## Exercise #2:
Now it's your turn. Use the code snippets from above to build out a *weighted* kNN function that takes in `specimen`, `training_data`, and `k` and outputs the prediction. 

In [50]:
import numpy

# weighted kNN
def w_knn(specimen, training_data, k):
  # calculate the distance
  data_copy = training_data.copy()
  data_copy['distance_to_new'] = numpy.sqrt((specimen['petal length']-data_copy['petal length'])**2
  + (specimen['sepal length']-data_copy['sepal length'])**2
  + (specimen['petal width']-data_copy['petal width'])**2
  + (specimen['sepal width']-data_copy['sepal width'])**2)
  
  # calculate the weights (remember, the weights are 1/d^2)
  data_copy['weights'] = 1/(data_copy['distance_to_new']**2)

  # find the closest k
  sorted_weight = data_copy.sort_values(['weight'], ascending = False)
  top_weight = sorted_weight[0:k] 

  # use groupby to sum the weights of each species in the closest k
  closest = top_weight.groupby(['species'])['weights'].sum()
  sorted = closest.
  # return the class that has the larger sum of weights
  return closest_sum.max()



In [ ]:
unknown_iris = {}
unknown_iris['petal length'] = 5.1
unknown_iris['sepal length'] = 7.2
unknown_iris['petal width'] = 1.5
unknown_iris['sepal width'] = 2.5
# 5-NN 
print(w_knn(unknown_iris,iris_data,5))
